In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline

In [2]:
from IPython.core.display import display, HTML

# 전체 구간을 넓게
display(HTML("<style>.container { width:90% !important; }</style>"))

# 각 컬럼 width 최대로
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 100)

C:\Users\kwon\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


# EDA

In [3]:
df_train = pd.read_csv('./data/train.csv').drop(['id', 'relevance_variance'], axis=1)

In [4]:
df_train.head()

,query,product_title,product_description,median_relevance
0,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,"Red satin accent pillow embroidered with a heart in black thread. 8"" x 8"".",1
1,led christmas lights,Set of 10 Battery Operated Multi LED Train Christmas Lights - Clear Wire,"Set of 10 Battery Operated Train Christmas Lights Item #X124210 Features: Color: multi-color bulbs with matching train light covers / clear wire Multi-color consists of red, green, blue and yellow bulbs Number of bulbs on string: 10 Bulb size: micro LED Spacing between bulbs: 6 inches Lighted length: 4.5 feet Total length: 5.5 feet 12 inch lead cord Additional product features: LED lights use 90% less energy Cool to the touch If one bulb burns out, the rest will stay lit Lights are equipped with Lamp Lock feature, which makes them replaceable, interchangeable and keeps them from falling out Requires 3 ""AA"" batteries (not included) Convenient on/off/timer switch located on battery pack Timer function on battery pack allows for 6 hours on and 18 hours off Cannot connect multiple sets together UL listed for indoor use only Train dimensions: 1.5""H x 1.8""W x .5""D Material(s): plastic/wire/acrylic",4
2,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4
3,wine rack,"Concept Housewares WR-44526 Solid-Wood Ceiling/Wall-Mount Wine Rack, Charcoal Grey, 6 Bottle","Like a silent and sturdy tree, the Southern Enterprises Bird and Branch Coat Rack is an eye-catching addition to your home décor. This tree themed coat rack features strong branches with pinecone accents and a small bird perched at the top to give it a whimsical and welcoming appearance while still making it sturdy enough to hold your coats, hats, umbrellas and more. Whether it serves as a coat rack, a hat rack or a combination of the two, it’ll be a great space saver that gets appreciated for its graceful appearance.\nNumber of Hooks: 10\nFrame Material: Metal\nHardware Material: Metal\nDimensions: 69.5 "" H x 13.25 "" W x 13.25 "" D\nWeight: 12.0 Lb.\nAssembly Details: assembly required",4
4,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),"WTGR1011\nFeatures\nNickel base, 60,000 average hours, acrylic resin bulb material\nChristmas light bulb\nSteady dimmable replacement lamps\nNickel bases prevent corrosion in sockets\nWattage: 0.96 Watts\nVoltage: 130 Volts\nDimmable: Yes\nLight Source: LED\nBulb Shape Type: Candle\n\nColor Amber\nBulb Color: Amber",2


In [5]:
df_train['query'].value_counts()

wireless mouse             113
rachel ray cookware        91 
memory foam pillow         90 
bike lock                  84 
16 gb memory card          64 
                           .. 
longboard pads             13 
silicone toe separators    12 
8 ounce mason jars         10 
polo bear sweater          10 
dollhouse bathtub          8  
Name: query, Length: 261, dtype: int64

In [6]:
df_train['product_title'].value_counts()

Wintergreen Lighting Christmas LED Light Bulb (Pack of 25)                                                  12
Logitech Wireless Mouse M325                                                                                6 
George Boys Dress Sets                                                                                      5 
George Boys' 2 Piece Dress Shirt and Tie Set                                                                5 
Graham & Graham Men's Dress Shirt & Tie Set - Green Check                                                   4 
                                                                                                           .. 
Chef's Choice Classic Belgian Nonstick WafflePro                                                            1 
Lotus Tree Print Yoga Mat                                                                                   1 
adidas Condivo 14 Training Pants - Mens - Black/White                                                       1 
W

In [7]:
num_of_products_per_query = []
for q in df_train['query'].value_counts().index:
    num_of_products_per_query.append(len(df_train.groupby('query').get_group(q)['product_title'].unique()))

    
print('쿼리당 평균 제품수 : %d ' % np.mean(num_of_products_per_query))
print('편차 : %d' % np.std(num_of_products_per_query))

쿼리당 평균 제품수 : 37 
편차 : 12


+ 분석 목표
  + 검색어와 검색 결과의 관련성을 측정하는 할 수 있는 모델을 만든다
    + embedding -> classification
  + 쿼리 - 제품, 설명 쌍을 입력으로 받아서 사용자의 점수(만족도)를 예측한다
    + if-idf -> classification

  
+ 모델을 만드는 이유
  + 검색 서비스를 평가할 수 있다.
    + 검색에 대한 정확성을 테스트 할 수 있다.
  + 모델을 기반으로 검색어가 왔을때 높은 만족도를 줄수있는 제품(관련성이 높은 제품)을 상위에 제공할 수 있다.


+ 살펴볼 데이터
  1. 쿼리
  1. 중복된 데이터 o
  1. 쿼리별 데이터 o
    + rating o
  1. 제품별 데이터 o
    + rating o 
  1. 다른 쿼리에서 같은 제품(있다면) o
  1. 같은 제품의 다른 설명
  
같은 쿼리, 제품명에서 다른 설명을 다른 데이터로 봐야하는가?

**해보고 싶은것**
+ embedding $\rightarrow$ clustering $\rightarrow$ topicmodeling 
 + word(sentence) embedding으로 vecterizing
 + query clustering : rating이 낮을경우 다른 검색어 추천
 + product clustering : rating이 애매할경우 다른 제품 추천

#### 쿼리

1. 무선 마우스(wireless mouse)
1. rachel ray조리도구(rachel ray cookware )
1. 메모리 폼 베개(memory foam pillow )
1. 자전거 자물쇠(bike lock)
1. 16기가 메모리 카드(16 gb memory card )
1. 로드 바이시클(road bicycle )
1. 블루투스 스피커(bluetooth speaker)
1. panasonic 면도기(shaver panasonic )
1. full tang knife(베어그릴스가 쓰는칼?)
1. 믹서기(kitchenaid mixer )
1. 주철 프라이팬(cast iron skillet)
1. usb-hdml 케이블(micro usb to hdmi )
1. 골드 드레스(gold dress )
1. 프로젝터(projector)
1. 스테인리스 목걸이(steel necklace )
1. 갤럭시 폰케이스(samsung galaxy phone case )
1. 더블 유모차(double stroller )
1. 디카(digital camera )
1. 바닐라 향 향수(vanilla scented perfumes )
1. 벽시계(wall clocks )
1. 공유기(라우터)(routers )
1. 냄비와 프라이팬 세트(pots and pans set )
1. 흰 드레스(white dress )
1. 닌자 거북이(teenage mutant ninja turtles )
1. 토스터 오븐(toaster oven )
1. 헬로 키티(hello kitty )
1. 커피머신(single serve coffee maker )
1. 어세신 크리드(assassinss creed )
1. 아로마 디퓨저(aroma diffuser )
1. ∞모양 스카프(infinity scarf )
1. face cream(로션)
1. 스프링 노트(notebook paper )
1. 주먹왕 랄프(wreck it ralph )
1. 밥솥(rice cooker )
1. 와인선반(wine rack )
1. 쿠진아트(회사명) 커피 메이커(cuisinart coffee maker)
1. 아이크림(eye cream )
1. 코코아 버터(cocoa butter)
1. 높은 의자(high chairs )
1. 이비노(회사명) 샴푸(aveeno shampoo )
1. 샤워가운(로브)(robe )
1. 하이힐(high heels shoes )
1. (닌텐도) 위(wii)
1. 네스프레소 커피(coffee for nespresso )
1. 새 먹이 피더(bird seed feeder)
1. 전기 철판(electric griddle)
1. 샌프란시스코 포티나이너스(럭비팀명)(san francisco 49ers)
1. 휴대용 하드 드라이버(portable hard drive)
1. 피츠버그 파이리츠(야구팀)(pittsburgh pirates)
1. 캔들 랜턴(candle lantern)
1. 큐리그(회사명) 커피메이커 (keurig coffee maker )
1. 토마스 기차(thomas the train )
1. 남자 셔츠(mens dress shirts )
1. 서랍(storage drawers)
1. iphone 4 case 
1. 긴 댄스 파티 드레스(long prom dress)
1. 그린베이 패커스(풋볼팀)(green bay packers )
1. 퀸 이불 세트(queen comforter sets )
1. galaxy note 3 
1. 압력솥(pressure cooker )
1. macbook case 13 case 
1. iphone 5 
1. (큐리그)커피 인듯?(k cups )
1. 여성 운동복 (workout clothes for women)
1. 탄산수 제조기(soda stream)
1. 강아지용 벼룩, 진드기 제거제(flea and tick control for dogs)
1. 와플 메이커 (waffle maker)
1. 폴라로이드 카메라 (polaroid camera) 
1. 서랍 정리함(drawer organizer) 
1. (legos) 
1. (aqua shoes) 
1. (dresses for women) 
1. 카고바지(cargo shorts) 
1. 편광 선글라스 (sunglasses polarized) 
1. 카키 바지(khaki pants)
1. 주방 수도꼭지(kitchen faucet) 
1. 한칸자리 쇼파?(storage ottoman) 
1. 파이렉스(유리 반찬통 만드는 회사)(pyrex) 
1. 캔들 샹들리에(candle chandelier)
1. (playstation 4) 
1. (laptop lenovo)
1. 전자레인지(microwave) 
1. 블랜더(음식 가는 용품)(blender) 
1. (toy trucks) 
1. 식탁보(table runner) 
1. 헤어 악세서리(머리핀 같은거)(hair accessories) 
1. 필립스 커피 메이커(phillips coffee maker)
1. 천 원통형 가방(duffle bag) 
1. 바비 (인형)(barbie)
1. 딘(회사명) 기타(dean guitar) 
1. (batman) 
1. plantronics(화사명) 유선 헤드셋(plantronics corded headset)
1. 가죽 남성 서류 가방(leather mens briefcase) 
1. 아이스크림 메이커(ice cream maker) 
1. 침대보(bedspreads) 
1. 어린이용 자전거(bike for children) 
1. 딘(회사명) 기타(dean guitar)
1. 방수 자켓(바막?)(rain jacket) 
1. 단추 있는 셔츠(button down shirt) 
1. 침낭(sleeping bags) 

1. 하와이안 셔츠(hawaiian shirt) 
1. 벽 글자 장식?(bridal shower decorations)
1. 커피 글라인더(coffee grinder) 
1. 전구(light bulb)
1. 스포츠 브라(sports bra) 
1. 플라넬(직물) 침대 시트(flannel sheets)
1. 모자 걸이(hat rack) 
1. 샘소나이트(화사명)(samsonite) 
1. 주방용 러그(kitchen rugs) 
1. 요가 바지(yoga pants) 
1. ps2 컨트롤러 USB(ps2 controller usb)
1. 목 베개(neck pillow) 
1. 청소 로봇(cleaning robot) 
1. 나이키 신발(nike shoes)
1. 냉장고(refrigirator) 
1. 드레스 셔츠(dress shirts)
1. 스케쳐스(회사명)여자 신발 (skechers womens shoes)
1. 무선 청소기(stick vacuum) 
1. 카우보이 부츠(cowboy boots) 
1. 은 목걸이(silver necklace) 
1. 노란색 드레스(yellow dress) 
1. fossil(회사명)여자 손목시계(watch women fossil)
1. 전기 담요(electric blanket)
1. 남성 가죽 셔츠(mens leather shirt) 
1. (lego star wars) 
1. (rain boot) 
1. 할리데이비슨(회사명, 오토바이)(harleydavidson) 
1. 골프 클럽(golf clubs) 
1. 목에 거는거?(dr who lanyard)
1. 스웨터 드레스(sweater dress) 
1. 야외 테이블 커버(outdoor table cover) 
1. (tv) 
1. 접시 타올(dish towels) 
1. 꽃 구근?(flower bulbs) 
1. 세 바퀴 자전거(three wheeled bike) 
1. 차장 스티커(car window sticker) 
1. 덕 다이너스티(tv 리얼리티 방송)(duck dynasty) 
1. like 에코백(tote bag) 
1. 빅토리녹스(회사명) 나이프(knife victorinox) 
1. 초코 틀(chocolate molds)
1. 금속 선반(metal lathe) 
1. 구찌 향수(gucci guilty intense women)
1. 남친 청바지(boyfriend jeans) 
1. 플스4 비디오 게임(videogames ps4) 
1. DC 슈즈(회사명) 블랙(dc shoes black)
1. 충전식 베터리(rechargable batteries) 
1. 뉴욕 양키스(야구팀)(yankees)
1. 마당 파라솔(patio umbrellas) 
1. 마당 식탁(patio tables)
1. 타블렛 악세서리(tablet accessories) 
1. silver jeans회사명(silver jeans) 
1. 위 게임패드(wii gamepad) 
1. 샷건 총알 파우치(shotgun shell pouch)
1. 테이블 시계(table clock)
1. 팬슬 스커트(A라인 스커트 같은거?)(pencil skirt) 
1. 심박수 모니터(polar heart rate monitor)
1. 지포 라이터(zippo)
1. 두꺼비집?(power surge protector)
1. 무릎 아래까지 오는 양말(gold toe socks)
1. 스펙(회사명) 아이폰 5 케이스(speck iphone 5 case) 
1. 유아 샌들(toddler sandals) 
1. 스노우 부츠(snow boots)
1. 음료 펀치 보울(punch bowls)
1. 운동할때 끼는 장갑(gym gloves) 
1. ???(brett favre ny titans jersey blue) 
1. 파란색 미니 냉장고(blue mini fridge)
1. fitbit(회사명) 밴드(fitbit charge) 
1. 헤드폰 이어 버즈(headphones earbuds) 
1. 리바이스(levis)
1. 배너티 페어(회사명) 브라(vanity fair bras) 
1. 여자 할로윈 의상(girls halloween costumes)
1. (yoga mat)
1. 아이패드2 튼튼한 케이스(ipad 2 heavy duty case) 
1. 태양열 배터리 충전기(solar battery charger) 
1. 비츠 일렉트로닉스(닥터드레) 해드폰(beats headphones) 
1. (mac book air cover) 
1. (led monitor) 
1. 비게 커버(decorative pillows) 
1. 홀마크(회사명)웨딩 커플 장식입상(hallmark wedding couple figurine)
1. 나이키 슬리퍼(nike flip flops)
1. 무예 해드 기어(martial art headgear) 
1. (spiderman) 
1. 크로스 백(cross strap backpack)
1. 오리새(식물)(cat grass) 
1. 햇빛 차단 설글라스(ray ban sunglasses)
1. (ps3 memory card)
1. (converse high tops) 
1. 휴대용 플스(playstation vita system)
1. bluesky(회사명) 젤 매니큐어(bluesky gel nail polish) 
1. (회사명)주거용 블렌더를 판매(blendtec)
1. 튀김기(deep fryers)
1. (golf shoes) 
1. (green laser pointer) 
1. (converse low top)
1. 리바이스 청바지(levis 505)
1. (coffee cup) 
1. (nike womens) 
1. (dual headset splitter) 
1. 회사명 남자 티셔츠(hollister polo) 
1. (silver ring) 

1. (adidas pants) 
1. 재사용 가능 빨대(reusable straws) 
1. 해골 포스터(해부학이나 그림?)(skeleton poster) 
1. 늑대 인간 의상(werewolf costume)
1. 맥주잔(beer stein) 
1. 빅토리아 시크릿(회사명)pink shorts(victoria 
1. 씨게이트(회사명)2tb 하드디스크(seagate 2tb hard disk) 
1. 멀티 폰 충전기(multiple phone charger) 
1. 러버메이드(가정용품회사)회전 탁자?(음악용품 아님)(rubbermaid turntables) 
1. 영화 호빗 확장판(bluray hobbit extended) 
1. 500기가 외장 하드(extenal hardisk 500 gb) 
1. 세이코(시계 회사)시계(seiko monster) 
1. 구피 영화(an extremely goofy movie)
1. 전자 책 리더(kobo ereader)
1. 마인크래프트 검(minecraft diamond sword)
1. 후지 바이크(자전거 용품 회사)셔츠(fuji bike shirt) 
1. 닌자 거북이 양말(ninja turtle socks)
1. 빅토리아 시크릿 lace gown(victorias secret lace gown)
1. 뉴트리 시스템(회사명)체중 감량 제품(nutri system) 
1. 자동차 방전 점프 스타터(car jump starter) 
1. 무릎까지 오는 스노우 부츠(over the knee snow boots) 
1. 아이폰 케이스 OtterBox(회사명)(apple iphone 32 gb otterbox) 
1. tupperware(회사명) water bottle lime (tupperware water bottle lime) 
1. k cups(donut shoppe k cups) 
1. 남자 다이아 귀걸이(men earrings diamond squared) 
1. 오클리(회사명) 선글라스(oakley polarized radar) 
1. (nike shox) 
1. 애니메 목걸이(anime necklace) 
1. (small brass bow) 
1. 볼컴(회사명) 반바지(volcom short) 
1. 오클리 선글라스(oakley radar) 
1. (fifa 15) 
1. 롱보드(타는거) 패드(보호대)(longboard pads) 
1. 실리콘 발가락 분리기? 발가락에 끼는거(silicone toe separators)
1. (polo bear sweater) 
1. 8 oz 메이슨 병 (8 ounce mason jars) 
1. 인형집 욕조(dollhouse bathtub) 

1. 로런 콘래드(디자이너) 드레스(lc lauren conrad dresses) 
1. (ps3 wireless controller) 
1. (yankee candle) 
1. KitchenAid(회사명) 유리 보울(kitchenaid glass bowl) 
1. ecco(회사명)신발(ecco shoes)
1. (white jeans) 
1. 크록스(회사명)고무 신(crocs) 
1. 야구 화(baseball cleats) 
1. (memory foam mattress topper)
1. (baseball photo frame) 
1. 아디다스 향수(adidas fragance)
1. 반스 백팩(vans backpack) 
1. 유리구슬(glass marbles) 
1. 반짝이 담는 병(glitter vials)

#### 중복된 데이터

In [8]:
df_train[['query', 'product_title', 'product_description']].duplicated().sum()

145

In [9]:
df_train[['query', 'product_title', 'product_description', 'median_relevance']].duplicated().sum()

100

####  쿼리별 데이터

In [10]:
querys = iter(df_train['query'].unique())

In [11]:
df_train[df_train['query'] == next(querys)]

,query,product_title,product_description,median_relevance
0,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,"Red satin accent pillow embroidered with a heart in black thread. 8"" x 8"".",1
797,bridal shower decorations,Personalized Bridal Shower Banner 36,Hang our Personalized Bridal Shower Banner to announce a bridal shower or to post well wishes to the bride-to-be! Our vinyl banners feature a pink rose design and your wording.,4
938,bridal shower decorations,Hers Accent Pillow - White,NaN,1
1234,bridal shower decorations,DDI 1122249 Bridal Shower Party 192 Piece Shipper Case Of 192,Bridal Shower Party Decor - 192 Piece Shipper 9 assorted decorations12 piece Invitation Cards10 piece Fountain Centerpiece 24 piece Confetti26 piece Letter Banner24 piece Honeycomb Centerpiece24 piece Honeycomb Bell24 piece Ribbon Centerpiece30 piece Swir SKU: DLRDY241074,1
1478,bridal shower decorations,"Bride to Be"" Floral Banner - Coral""","details\nUse as a photo prop or deocration, white cards spelling out ""Bride to Be"" printed in coral and white with a whimsical design. Include a 3/8"" wide by 70"" long white satin ribbon. Customer assembly required.\nPackage Quantity: 1",2
1494,bridal shower decorations,"The New Mr"" and ""Mrs"" Banner Set - Brown""","Five piece pennant banner with wording ""The Future Mr. & Mrs."". Made of 100% natural jute fabric. Each pennant measures 10"" x 8"". Includes jute cord for assembly. Embroidered in black and red thread.",4
1719,bridal shower decorations,Bridal Shower Personalized Photo Banner 48,NaN,2
1721,bridal shower decorations,"Blue Stork Baby Shower Hanging Decorations, 3pk","Add some style to your beautiful baby shower with our Blue Stork Baby Shower Hanging Decorations. Featuring shiny, blue foil swirls with stork and baby bear cut outs, these adorable party decorations are a simple way to add style to your upcoming baby shower. Coordinate these Blue Stork Baby Shower Hanging Decorations with other Blue Stork Baby Shower party supplies or combine with matching solid color or baby shower themed party decorations for a sweet baby shower atmosphere. Blue Stork Baby Shower Hanging Decorations extend to 36 inches long and are sold three to a package.\n\nBlue Stork Baby Shower Hanging Decorations, 3pk:\nPackage of 3 Blue Stork Baby Shower Party Decorations\nHanging Party Decorations extend to 36"" long\nHang from walls and ceiling to add style to your baby shower party\nBlue Stork Baby Shower Hanging Decorations are made of plastic and durable paper stock\nCoordinate with other Blue Stork Baby Shower party supplies and solid color party decorations",2
1802,bridal shower decorations,DDI 1122249 Bridal Shower Party 192 Piece Shipper Case Of 192,Bridal Shower Party Decor - 192 Piece Shipper 9 assorted decorations12 piece Invitation Cards10 piece Fountain Centerpiece 24 piece Confetti26 piece Letter Banner24 piece Honeycomb Centerpiece24 piece Honeycomb Bell24 piece Ribbon Centerpiece30 piece Swir,4
2357,bridal shower decorations,Bridal Shower Personalized Photo Banner 30,Celebrate a bridal shower with our Bridal Shower Personalized Photo Banner that features your own wording and photo on a pretty pink background.,4


#### 쿼리 & rating 데이터

In [12]:
def q_r_iter():
    n=0
    while n <= 261 * 4:
        yield df_train['query'].unique()[n//4], (n%4)+1
        n += 1
        
q_r = q_r_iter()

In [13]:
q, r = next(q_r)
print('query : %s \nrating : %d' % (q, r))
df_train[(df_train['query'] == q) & (df_train['median_relevance'] == r)]

query : bridal shower decorations 
rating : 1


,query,product_title,product_description,median_relevance
0,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,"Red satin accent pillow embroidered with a heart in black thread. 8"" x 8"".",1
938,bridal shower decorations,Hers Accent Pillow - White,NaN,1
1234,bridal shower decorations,DDI 1122249 Bridal Shower Party 192 Piece Shipper Case Of 192,Bridal Shower Party Decor - 192 Piece Shipper 9 assorted decorations12 piece Invitation Cards10 piece Fountain Centerpiece 24 piece Confetti26 piece Letter Banner24 piece Honeycomb Centerpiece24 piece Honeycomb Bell24 piece Ribbon Centerpiece30 piece Swir SKU: DLRDY241074,1
4206,bridal shower decorations,Sand Ceremony Shadow Box - Heart Design,"Commemorate your beautiful union with the Sand Ceremony Shadowbox. Featuring an interlocking Heart Design; this classic symbol is sure to always bring a smile to your face when you remember the day your 2 hearts joined to make one, never to be separated again. Includes two pouring vases. Sand not included.",1
4923,bridal shower decorations,Planet Cake Celebrate: Cake Making for All Occasions (Paperback),"Paris Cutler has always been passionate about cake making and loves the joy and amazement that cakes evoke in friends and customers. Paris bought Planet Cake, a small cake-decorating studio in Sydney, in 2003, and has steered it to international renown. As well as producing great works of cake art for celebrities such as Nicole Kidman, Rihanna, Celine Dion and gourmets alike, Paris runs popular cake-decorating classes through her schools across Australia, which attract students from around the world.\nBe the star of the party with Paris Cutlers inspiring guide to cake making for birthdays, fundraisers, bridal showers and more. With playful ideas and detailed instructions, even complete beginners to cake decorating will find themselves creating little masterpieces. From Cheeky Monkey and Polka Dot Cupcakes for the school fete to Freddy and Girly Bear Cupcakes for a children's party, Planet Cake Celebrate will have a cake for every occasion! The book includes more than 20 cupcake and mini fudge cake designs for all occasions. It also has detailed advice on how you can host events and how to prepare for a cake-making party, advice on taking cakes home, templates, techniques and troubleshooting tips.",1
5095,bridal shower decorations,"Pink Stork Baby Shower Mini Honeycomb Decorations, 4pk","Sweeten up your table with our adorable Pink Stork Baby Shower Decorations. Measuring 6"" high and featuring a charming stork holding a baby bear, these sturdy, cardboard table centerpieces are the perfect addition to your baby shower party decorations. Coordinate them with other Pink Stork party supplies and solid color or baby shower themed party decorations to personalize your baby shower. These mini honeycomb decorations are sold in a 4-pack and measure 6"" high.",1
5123,bridal shower decorations,Decorative Egg Ring Box - Silver,"details\nPackage Quantity: 1\nFeatures: sparkles, Reusable\nMaterial: Metal\nCare and Cleaning: Wipe Clean With a Damp Cloth\nDimensions: 3.75 "" H x 2.0 "" W x 2.0 "" L\nWeight: 0.7 gsm\nBattery: no battery used\nother Info.\n Online Item #: 16461129\n Store Item Number (DPCI): 053-06-1490\n Imported",1
5128,bridal shower decorations,& Accent Pillow - Beige,"Natural-color linen accent pillow with ""&"" embroidered in black thread. 8"" x 8"".",1
5779,bridal shower decorations,His Accent Pillow - White,"White satin accent pillow embroidered with ""His"" in black thread. 12"" x 12"".",1
6176,bridal shower decorations,Mr. & Mrs. Throw Pillow Set,"ITEM#: 15765888\nPay homage to your newlywed status all year long with this Mr. & Mrs. throw pillow set. Fashioned in a crisp white canvas covering, the decorative accent pillows will look great just about anywhere. \n\nQuantity: Set of two (2) pillows\nDimensions: 12 inches high x 12 inches wide each\nColor: White\nFill: Polyester filling\nCover: Canvas casing\nIncludes per

#### 제품별 데이터

In [14]:
title = iter(df_train['product_title'].value_counts().head(385).index)

In [15]:
df_train.groupby('product_title').get_group(next(title))

,query,product_title,product_description,median_relevance
4,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),"WTGR1011\nFeatures\nNickel base, 60,000 average hours, acrylic resin bulb material\nChristmas light bulb\nSteady dimmable replacement lamps\nNickel bases prevent corrosion in sockets\nWattage: 0.96 Watts\nVoltage: 130 Volts\nDimmable: Yes\nLight Source: LED\nBulb Shape Type: Candle\n\nColor Amber\nBulb Color: Amber",2
206,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),WTGR1011,2
2002,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),"WTGR1008\nFeatures\nNickel base, 60,000 average hours, acrylic resin bulb material\nChristmas light bulb\nSteady dimmable replacement lamps\nNickel bases prevent corrosion in sockets\nWattage: 0.96 Watts\nVoltage: 130 Volts\nDimmable: Yes\nLight Source: LED\nBulb Shape Type: Candle",4
2222,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),"Características: base -Nickel, 60000 horas promedio, materiales bombilla de resina acrílica. -Navidad bombilla. Lámparas de repuesto regulables -Steady. Bases -Nickel evitar la corrosión en los zócalos. Potencia: -0,96 Watts. Voltaje: -130 voltios. Regulable: -Sí. Fuente de luz: del LED. Bombilla Tipo Forma: -Candle. Bombillas fluorescentes GU10 luces halógenas Comercial ámbar azul fresco Oro Verde Multicolor Rosa Púrpura Rojo Caliente blanco 10W 130V 199V Regulable Vela Bajo días de fiesta, navidad regalo regalos para los muchachos",4
2925,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),NaN,4
3312,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),NaN,4
3740,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),NaN,3
4121,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),NaN,2
8556,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),"WTGR1011\nFeatures\nNickel base, 60,000 average hours, acrylic resin bulb material\nChristmas light bulb\nSteady dimmable replacement lamps\nNickel bases prevent corrosion in sockets\nWattage: 0.96 Watts\nVoltage: 130 Volts\nDimmable: Yes\nLight Source: LED\nBulb Shape Type: Candle",3
9239,light bulb,Wintergreen Lighting Christmas LED Light Bulb (Pack of 25),NaN,4


#### 제품 & rating 데이터

In [16]:
def p_r_iter():
    n=0
    while n <= 9708 * 4:
        yield df_train['product_title'].value_counts().index[n//4], (n%4)+1
        n += 1
        
p_r = p_r_iter()

In [17]:
p, r = next(p_r)
print('title : %s \nrating : %d' % (p, r))
df_train[(df_train['product_title'] == p) & (df_train['median_relevance'] == r)]

title : Wintergreen Lighting Christmas LED Light Bulb (Pack of 25) 
rating : 1


,query,product_title,product_description,median_relevance


####  서로다른 쿼리에서 검색된 같은 제품

In [18]:
p_list = []
for p in df_train['product_title'].value_counts().head(385).index:
    if len(df_train[df_train['product_title'] == p]['query'].unique()) >= 2:
        p_list.append(p)
        
len(p_list)

129

In [19]:
p_r_iter = iter(p_list)

In [20]:
df_train.groupby('product_title').get_group(next(p_r_iter)).sort_values(by='query')

,query,product_title,product_description,median_relevance
51,dress shirts,George Boys' 2 Piece Dress Shirt and Tie Set,"This George Boys 2 Piece Dress Shirt and Tie Set will make your boy look so smart and sharp. With comfortable design and impeccable match, this 2-piece set will make your boy look great. Each piece is also great for mix and match with his existing closet. Choose from assorted options.",3
5683,dress shirts,George Boys' 2 Piece Dress Shirt and Tie Set,"This George Boys 2 Piece Dress Shirt and Tie Set will make your boy look so smart and sharp. With comfortable design and impeccable match, this 2-piece set will make your boy look great. Each piece is also great for mix and match with his existing closet. Choose from assorted options.",4
448,long prom dress,George Boys' 2 Piece Dress Shirt and Tie Set,"Get little guys ready for a special occasion with this George Boys' 2-Piece Dress Shirt and Tie Set. These items are a good choice for seasonal events, pictures, concerts and more. The set includes a tie and a long-sleeve button-down shirt. The clip-on tie is so cute and convenient. The long-sleeve shirt features a button-up style with a collar to create a dressed-up look. It is made of a cotton-poly blend for breathable comfort and durability. A front pocket offers a dapper accent. Pair this boys' shirt and tie set with coordinating dress slacks to complete the look. Select multiple sets for mix-and-match styling.",1
8434,long prom dress,George Boys' 2 Piece Dress Shirt and Tie Set,"Your little boy will look dressed up and handsome for his next big event when he wears the George Boys' 2-Piece Dress Shirt and Tie Set. It comes with a clip-on tie that is cute and convenient. The boys' shirt and tie set is designed so the pieces go well together, and they coordinate well with any pair of dress pants. The button-up style shirt has long sleeves and a button closure at the cuffs, and it has one front pocket. It is made from a comfortable blend of cotton and polyester. The tie is 100 percent polyester. The set can be machine washed for easy care. Now your little boy will have the chance to wear something nice when he dresses up for a formal occasion.",1
9181,long prom dress,George Boys' 2 Piece Dress Shirt and Tie Set,NaN,1


In [55]:
submission_proba = pd.read_csv('./submission/proba.csv')

In [56]:
submission_proba1 = pd.read_csv('./submission/submission_proba_eda_10.csv')

In [57]:
submission_proba2 = pd.read_csv('./submission/submission_proba_eda_200_20_7.csv')

In [58]:
submission_proba3 = pd.read_csv('./submission/submission_proba_oversampling_10.csv')

In [59]:
submission_proba4 = pd.read_csv('./submission/submission_proba_over_230_10_7.csv')

In [ ]:
submission_proba3 = pd.read_csv('./submission/submission_proba_eda_over_200_15_7.csv')

In [ ]:
submission_proba3 = pd.read_csv('./submission/submission_proba_eda_over_150_10_7.csv')

In [60]:
submission = (submission_proba + submission_proba1 + submission_proba2 + submission_proba3 + submission_proba4)/5

In [4]:
submission = (submission_proba + submission_proba1)/2

In [11]:
submission = (submission_proba + submission_proba2)/2

In [52]:
submission = (submission_proba1 + submission_proba2)/2

In [49]:
submission = (submission_proba + submission_proba1 + submission_proba2)/3

In [61]:
submission = submission.astype({'Unnamed: 0': 'int32'})
submission.rename(columns={"Unnamed: 0": "id"}, inplace=True)
submission['prediction'] = submission[['0', '1', '2', '3']].idxmax(axis=1).astype('int64')+1
submission.drop(['0', '1', '2', '3'], axis=1, inplace=True)
submission.to_csv("./submission/submission_ensemble9.csv", index=False)